# Credit Risk Resampling Techniques

In [475]:
import warnings
warnings.filterwarnings('ignore')

In [476]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [477]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
dfreplaced=df['loan_status'].replace("low_risk", 0)

In [478]:
dfreplaced=dfreplaced.replace("high_risk", 1)

In [479]:
dfreplaced

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

# Split the Data into Training and Testing

In [480]:
# Create our features
X = df.drop(columns='loan_status')
X=pd.get_dummies(X)

# Create our target
y = df.loc[:,["loan_status"]].copy()

In [481]:
X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,1,0,0
...,...,...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600,0,1,0
77532,17700.0,10.662,80900,0.629172,11,2,50900,1,0,0
77533,17600.0,10.595,80300,0.626401,11,2,50300,0,0,1
77534,16300.0,10.068,75300,0.601594,10,2,45300,1,0,0


In [482]:
# Check the balance of our target values
y

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk
...,...
77531,high_risk
77532,high_risk
77533,high_risk
77534,high_risk


In [483]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [484]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [485]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset


In [486]:
# Scale the training and testing data


# Simple Logistic Regression

In [487]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [488]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9481182566723452

In [489]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  541,    59],
       [  102, 18682]])

In [490]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.90      0.99      0.87      0.95      0.89       600
   low_risk       1.00      0.99      0.90      1.00      0.95      0.91     18784

avg / total       0.99      0.99      0.90      0.99      0.95      0.90     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [491]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

# View the count of target classes with Counter
# YOUR CODE HERE
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train,y_train)
Counter(y_resampled)


Counter({'loan_status': 1})

In [492]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [493]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9481182566723452

In [494]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test,y_pred)

array([[  597,     3],
       [  111, 18673]])

In [495]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       600
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [496]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
from collections import Counter

Counter(y_resampled)




Counter({'loan_status': 1})

In [497]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [498]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9945453577512777

In [499]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  597,     3],
       [  111, 18673]])

In [500]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       600
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [501]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids 

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)
# View the count of target classes with Counter

Counter({'loan_status': 1})

In [502]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs',random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [503]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.9945453577512777

In [511]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred)

array([[  598,     2],
       [  114, 18670]])

In [512]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling alclassification_report_imbalancedf the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [513]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

# View the count of target classes with Counter
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [514]:
# Train the Logistic Regression model using the resampled data


model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [515]:
# Calculate the balanced accuracy score

balanced_accuracy_score(y_test, y_pred)

0.9952988358886996

In [516]:
# Display the confusion matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test,y_pred)

array([[  598,     2],
       [  114, 18670]])

In [517]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

    Combination(over and under)

2. Which model had the best recall score?

    Combination

3. Which model had the best geometric mean score?

    Undersampling/combination 
   

In [518]:
1.

1.0

In [519]:
2.

2.0

In [ ]:
3.